# Chạy thực nghiệm dự án ViT5‑VQA trên bộ dữ liệu OpenViVQA

Notebook này hướng dẫn cách clone repo `M5ViVQA`, cài đặt các phụ thuộc cần thiết,
chuẩn bị dữ liệu và huấn luyện mô hình ViT5‑VQA (bao gồm cả biến thể có MoE) trên
bộ dữ liệu OpenViVQA. Các bước được thiết kế để bạn có thể chạy trên nền tảng
[Kaggle Notebook](https://www.kaggle.com/) với GPU.


## 2. Clone repository và chuẩn bị dự án

Chúng ta sẽ clone toàn bộ repository `M5ViVQA` từ GitHub và chuyển vào thư mục
làm việc. Nếu bạn đã fork dự án, hãy thay đổi URL cho phù hợp.


In [ ]:
# Clone repo (chứa toàn bộ mã nguồn tổ chức lại)
!git clone https://github.com/Kussssssss/M5ViVQA.git
%cd M5ViVQA

# Cài đặt các thư viện dựa trên requirements.txt
%pip install --quiet -r requirements.txt

# Hiển thị cấu trúc thư mục để tham khảo
!ls -R . | sed -n '1,200p'


## 3. Tải bộ dữ liệu OpenViVQA

Dự án đi kèm script `scripts/download_data.py` để tải và giải nén bộ dữ liệu
OpenViVQA từ Google Drive. Bạn có thể chạy script này như sau. Hãy đảm bảo
bạn có đủ dung lượng lưu trữ (~2 GB) và kết nối internet.

Ngoài ra, nếu bạn muốn tải dữ liệu trực tiếp qua Kaggle API, hãy chỉnh sửa
phần này cho phù hợp (ví dụ: `kaggle datasets download ...`).


In [ ]:
# Sử dụng script mới để tự động tải dữ liệu OpenViVQA từ Google Drive
!python scripts/download_data_openvivqa.py

# Sao chép dữ liệu ra thư mục data/openvivqa để tương thích với cấu hình
!mkdir -p data && cp -r /kaggle/working/OpenViVQA data/openvivqa || true

# Kiểm tra thư mục dữ liệu
!ls data/openvivqa


### Dữ liệu mẫu (tuỳ chọn)

Nếu bạn chỉ muốn chạy thử nghiệm nhanh hoặc không có đủ tài nguyên GPU,
script `scripts/create_sample_data.py` có thể tạo bộ dữ liệu nhỏ với vài ảnh
màu cơ bản. Bạn có thể sử dụng cấu hình mẫu trong `config.py` (`SAMPLE_CONFIG`).


In [ ]:
# Tạo dữ liệu mẫu (tuỳ chọn)
!python scripts/create_sample_data.py --output_dir sample_data

# Kiểm tra dữ liệu mẫu
!ls sample_data


## 4. Huấn luyện mô hình

Sử dụng script `main.py` (wrapper) để huấn luyện mô hình. Bạn có thể chọn giữa cấu
hình đầy đủ (`FULL_CONFIG`) và cấu hình mẫu (`SAMPLE_CONFIG`) được định nghĩa
trong `config.py` bằng tham số `--config`. Trong ví dụ dưới đây chúng ta sử dụng
cấu hình mẫu để rút ngắn thời gian chạy. Để huấn luyện trên dữ liệu thật, thay thế `sample` bằng `full`.


In [ ]:
# Huấn luyện mô hình bằng config mẫu (chạy ~vài phút trên GPU)
!python main.py --config sample


## 5. Đánh giá mô hình trên tập kiểm tra

Sau khi huấn luyện xong, mô hình và tokenizer sẽ được lưu trong thư mục `output_dir`
theo cấu hình. Dưới đây là ví dụ gọi lại huấn luyện viên để sinh câu trả
lời và tính toán các chỉ số đánh giá sử dụng hàm `compute_vqa_metrics`.


In [ ]:
import json
from pathlib import Path
from openvivqa.evaluation.metrics import compute_vqa_metrics
import torch

# Cấu hình đường dẫn theo output_dir trong config
output_dir = Path('sample_results')  # hoặc 'vit5-vqa-results' nếu dùng cấu hình full
model_dir = output_dir / 'checkpoint-best'

# Load mô hình (tokenizer nằm trong model_dir)
from openvivqa.models import ViT5VQAModel
model = ViT5VQAModel.from_pretrained(model_dir)
tokenizer = model.tokenizer
model.eval()

# Load dữ liệu test
test_data = json.load(open('sample_data/test.json'))

preds, labels = [], []
for item in test_data:
    question = item['question']
    answer = item['answer']
    # Bạn cần trích xuất đặc trưng ảnh phù hợp với pipeline (bỏ qua trong ví dụ này)
    # Ở đây chúng ta giả sử ảnh đã được xử lý hoặc sử dụng placeholder
    raise NotImplementedError('Cần triển khai xử lý ảnh cho dữ liệu thực để sinh câu trả lời')

# Khi có preds và labels, gọi:
# metrics = compute_vqa_metrics(preds, labels)
# print(metrics)
